In [1]:
# import Pkg;
# Pkg.add("POMDPSimulators")
# Pkg.add("POMDPs")
# Pkg.add("POMDPTools")
# Pkg.add("QuickPOMDPs")
# Pkg.add("MCTS")
# Pkg.add("DiscreteValueIteration")
# Pkg.add("POMDPModels")
# Pkg.add("POMDPPolicies")
# Pkg.add("Distributions")
# Pkg.add("Combinatorics")
# Pkg.add("StaticArrays")
# Pkg.add("D3Trees")
# Pkg.add("Random")
# Pkg.add("FileIO")
# Pkg.add("JLD2")
# Pkg.add("TickTock")



In [2]:
# import Pkg; Pkg.add("DataFrames")

In [3]:
using POMDPs, POMDPTools, QuickPOMDPs, MCTS, DiscreteValueIteration, POMDPSimulators, POMDPModels, POMDPPolicies
using Distributions, Combinatorics, StaticArrays, D3Trees, Random
using FileIO, JLD2, TickTock

In [4]:
global Number_level = 4;
global Action_num = 4;

gamma=0.95
Transition_matrix = [
    0.8571 0.1429 0.0    0.0;
    0.0    0.8571 0.1429 0.0;
    0.0    0.0    0.8    0.2;
    0.0    0.0    0.0    1.0
]
T=zeros(Number_level,Number_level,3);
T[:,:,1].=Transition_matrix;
T[:,:,2].=Transition_matrix;
for j in 1:Number_level
    T[j,:,3].=Transition_matrix[1,:];
    end
T[Number_level,:,1].=0.0
T[Number_level,Number_level,1]=1.0
T[Number_level,:,2].=Transition_matrix[1,:]


4-element view(::Array{Float64, 3}, 4, :, 2) with eltype Float64:
 0.8571
 0.1429
 0.0
 0.0

In [5]:
T

4×4×3 Array{Float64, 3}:
[:, :, 1] =
 0.8571  0.1429  0.0     0.0
 0.0     0.8571  0.1429  0.0
 0.0     0.0     0.8     0.2
 0.0     0.0     0.0     1.0

[:, :, 2] =
 0.8571  0.1429  0.0     0.0
 0.0     0.8571  0.1429  0.0
 0.0     0.0     0.8     0.2
 0.8571  0.1429  0.0     0.0

[:, :, 3] =
 0.8571  0.1429  0.0  0.0
 0.8571  0.1429  0.0  0.0
 0.8571  0.1429  0.0  0.0
 0.8571  0.1429  0.0  0.0

In [6]:
Observation_matrix=[
    0.8 0.2 0.0 0.0;
    0.1 0.8 0.1 0.0;
    0.0 0.2 0.8 0.0;
    0.0 0.0 0.0 1.0
]
# Observation_matrix=[
#     0.7 0.2 0.1 0.0;
#     0.15 0.7 0.15 0.0;
#     0.1 0.2 0.7 0.0;
#     0.0 0.0 0.0 1.0
# ]

# Observation_matrix=[
#     0.65  0.25 0.1   0.0;
#     0.175 0.65 0.175 0.0;
#     0.1   0.25 0.65  0.0;
#     0.0   0.0  0.0   1.0
# ]

4×4 Matrix{Float64}:
 0.8  0.2  0.0  0.0
 0.1  0.8  0.1  0.0
 0.0  0.2  0.8  0.0
 0.0  0.0  0.0  1.0

In [7]:
NumberUnits=20
limit=12
failure_penalty = -1200.0;
maintenance_penalty = -100.0;
setup_cost = -300.0;
normal_operation = 0.0;
system_penalty=-2000

-2000

In [8]:
# import Pkg; Pkg.add("QMDP")

In [9]:
States=Array((1:Number_level))
Observations=Array((1:Number_level))

using POMDPs, QuickPOMDPs, POMDPModelTools, POMDPSimulators, QMDP

In [10]:
# In iterations, belief needs to be "manually" updated using updater(policy::Policy), 
# then call decision function using policy and belief state


In [11]:
SparseCat(Observations, Observation_matrix[1,:])

               SparseCat distribution           
     ┌                                        ┐ 
   1 ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 0.8   
   2 ┤■■■■■■■■■ 0.2                             
   3 ┤ 0.0                                      
   4 ┤ 0.0                                      
     └                                        ┘ 

In [12]:
Initialstates = zeros(Number_level)
Initialstates[1]=1.0
Initialstates

4-element Vector{Float64}:
 1.0
 0.0
 0.0
 0.0

In [13]:
Initialstate = SparseCat(States, Initialstates)

               SparseCat distribution           
     ┌                                        ┐ 
   1 ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 1.0   
   2 ┤ 0.0                                      
   3 ┤ 0.0                                      
   4 ┤ 0.0                                      
     └                                        ┘ 

In [14]:
# Initialobss = zeros(Number_level)
# Initialobss[1]=1.0
# Initialobss

In [15]:
# Initialobs = SparseCat(States, Initialobss)
Initialobs = SparseCat(Observations, Observation_matrix[1,:])

               SparseCat distribution           
     ┌                                        ┐ 
   1 ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 0.8   
   2 ┤■■■■■■■■■ 0.2                             
   3 ┤ 0.0                                      
   4 ┤ 0.0                                      
     └                                        ┘ 

In [16]:
multiunit = QuickPOMDP(
    states=States,
    actions=Array((1:Action_num)),
    observations = Observations,
    #initialstate = repeat(1:1,NumberUnits),
    initialstate = Initialstate,
    #initialobs = repeat(1:1,NumberUnits),
    initialobs = Initialobs,
    discount = gamma,

    transition = function(s,a)
        if a == 1 # NO REPAIR, NO SYSTEM FAILURE
            #crd = Categorical(T[s,:,1]);
            #s = rand(crd);
            return SparseCat(States, T[s,:,1])
        end
        if a == 2 # UNIT NOT REPAIRED UNLESS FAILURE BUT REPAIR IN SYSTEM
            if s == Number_level
                s = 1
                #crd = Categorical(T[s,:,2]);
                #s = rand(crd);
                return SparseCat(States, T[s,:,2])
            else
                #crd = Categorical(T[s,:,2]);
                #s = rand(crd);
                return SparseCat(States, T[s,:,2])
            end
        end
        if a == 3 # UNIT REPAIRED AND MAINTENANCE IN SYSTEM, NO SYSTEM FAILURE
            if s==Number_level
                s=1
                #crd = Categorical(T[s,:,3]);
                #s = rand(crd);
                return SparseCat(States, T[s,:,3])
            else
                s = 1
                #crd = Categorical(T[s,:,3]);
                #s = rand(crd);
                return SparseCat(States, T[s,:,3])
            end
        end
        if a == 4 # UNIT REPAIRED AND MAINTENANCE IN SYSTEM,SYSTEM FAILED
            if s==Number_level
                s=1
                #crd = Categorical(T[s,:,3]);
                #s = rand(crd);
                return SparseCat(States, T[s,:,3])
            else
                s = 1
                #crd = Categorical(T[s,:,3]);
                #s = rand(crd);
                return SparseCat(States, T[s,:,3])
            end
        end
    end,

    observation = function(a,s)
        return SparseCat(Observations, Observation_matrix[s,:])
    end,
        
    reward = function (s, a)
        if a == 1 # NO REPAIR, NO SYSTEM FAILURE
            if s == Number_level
                return system_penalty/(limit)
            else
                return normal_operation
            end
            #crd = Categorical(T[s,:,1]);
            #s = rand(crd);
        end
        if a == 2 # UNIT NOT REPAIRED UNLESS FAILURE BUT REPAIR IN SYSTEM, NO SYSTEM FAILURE PENALTY
            if s == Number_level
                #s = 1
                #crd = Categorical(T[s,:,2]);
                #s = rand(crd);
                return failure_penalty +  setup_cost/NumberUnits + normal_operation
            else
                # Here it doesn't matter whether systems fails or not, since C_b does not apply to functional units
                return normal_operation +  setup_cost/NumberUnits
            end
        end
        if a == 3 # UNIT REPAIRED AND MAINTENANCE IN SYSTEM, NO SYSTEM FAILURE PENALTY
            if s==Number_level
                #s=1
                #crd = Categorical(T[s,:,3]);
                #s = rand(crd);
                return failure_penalty +  setup_cost/NumberUnits + normal_operation
            else
                # Here it doesn't matter whether systems fails or not, since C_b does not apply to functional units
                return maintenance_penalty +  setup_cost/NumberUnits+ normal_operation 
            end
        end
        if a == 4 # UNIT REPAIRED AND MAINTENANCE IN SYSTEM,SYSTEM FAILED
            if s==Number_level
                #s=1
                #crd = Categorical(T[s,:,3]);
                #s = rand(crd);
                return failure_penalty +  setup_cost/NumberUnits + normal_operation + system_penalty/(limit)
            else
                #s = 1
                #crd = Categorical(T[s,:,3]);
                #s = rand(crd);
                return maintenance_penalty +  setup_cost/NumberUnits + normal_operation
            end
        end
    end,
    

)

QuickPOMDP{Base.UUID("5debbec2-f653-42ca-bffc-89021d95a8d2"), Int64, Int64, Int64, @NamedTuple{stateindex::Dict{Int64, Int64}, isterminal::Bool, obsindex::Dict{Int64, Int64}, states::Vector{Int64}, observations::Vector{Int64}, discount::Float64, initialobs::SparseCat{Vector{Int64}, Vector{Float64}}, actions::Vector{Int64}, observation::var"#2#5", actionindex::Dict{Int64, Int64}, initialstate::SparseCat{Vector{Int64}, Vector{Float64}}, transition::var"#1#4", reward::var"#3#6"}}((stateindex = Dict(4 => 4, 2 => 2, 3 => 3, 1 => 1), isterminal = false, obsindex = Dict(4 => 4, 2 => 2, 3 => 3, 1 => 1), states = [1, 2, 3, 4], observations = [1, 2, 3, 4], discount = 0.95, initialobs = SparseCat{Vector{Int64}, Vector{Float64}}([1, 2, 3, 4], [0.8, 0.2, 0.0, 0.0]), actions = [1, 2, 3, 4], observation = var"#2#5"(), actionindex = Dict(4 => 4, 2 => 2, 3 => 3, 1 => 1), initialstate = SparseCat{Vector{Int64}, Vector{Float64}}([1, 2, 3, 4], [1.0, 0.0, 0.0, 0.0]), transition = var"#1#4"(), reward = var"

In [17]:
initialstate(multiunit)

               SparseCat distribution           
     ┌                                        ┐ 
   1 ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 1.0   
   2 ┤ 0.0                                      
   3 ┤ 0.0                                      
   4 ┤ 0.0                                      
     └                                        ┘ 

In [18]:
# import Pkg; Pkg.add("NativeSARSOP")
using POMDPs
# using NativeSARSOP
using POMDPModels



In [19]:
ordered_states(multiunit)

4-element Vector{Int64}:
 1
 2
 3
 4

In [20]:
length(states(multiunit))

4

In [21]:
#import Pkg; Pkg.add("IncrementalPruning")

In [22]:
# using IncrementalPruning
# solver = PruneSolver() # set the solver

# policy = solve(solver, multiunit)

In [23]:
#value(policy,[0.1,0.8,0.1,0.0])

In [24]:
# solver = SARSOPSolver()
# policy = solve(solver, multiunit)
# # Calling observation

In [25]:
# value(policy,[0.0,0.0,0.0,1.0])

In [26]:
# value(policy,[0.1,0.1,0.8,0.0])

In [27]:
b=[[0.0, 0.2, 0.8, 0.0] for j in 1:NumberUnits]
belief=b[1]
# value(policy,belief)

4-element Vector{Float64}:
 0.0
 0.2
 0.8
 0.0

In [28]:
#transition function: transition(pomdp, state, action)
#return the transition distribution form the current state-action pair
transition(multiunit, 3,1)

               SparseCat distribution           
     ┌                                        ┐ 
   1 ┤ 0.0                                      
   2 ┤ 0.0                                      
   3 ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 0.8   
   4 ┤■■■■■■■■■ 0.2                             
     └                                        ┘ 

In [29]:
# observation function: observation(pomdp, action,statep)
# return the observation distribution
POMDPs.observation(multiunit,1,2)

               SparseCat distribution           
     ┌                                        ┐ 
   1 ┤■■■■ 0.1                                  
   2 ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 0.8   
   3 ┤■■■■ 0.1                                  
   4 ┤ 0.0                                      
     └                                        ┘ 

In [30]:
reward(multiunit, 2,2)

-15.0

In [31]:
s=1
new_a = 1
p_observation = POMDPs.observation(multiunit,new_a,s)


               SparseCat distribution           
     ┌                                        ┐ 
   1 ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 0.8   
   2 ┤■■■■■■■■■ 0.2                             
   3 ┤ 0.0                                      
   4 ┤ 0.0                                      
     └                                        ┘ 

In [32]:
# use pdf function to get prob of certain entry
new_obs=2
pdf(p_observation,new_obs)

0.2

In [33]:
function generative(s,a,rng=23) # calculate TRUE COMPONENT-LEVEL reward r_i(s_i,a_i) and get evolving states
    if a==1 # No maintenance in the system, no system failure
        crd=Categorical(T[s,:,1])
        s=rand(crd)
        r=normal_operation
        obs_distribution = SparseCat(Observations, Observation_matrix[s,:])
        obs = rand(obs_distribution)
        return (sp=s,r=r,o=obs)
    end

    if a==2 # Maintenance occurs in the system, no system failure, corrective maintenance for the unit
        if s==Number_level
            crd=Categorical(T[1,:,2])
            s=rand(crd)
            r=failure_penalty  + normal_operation
            obs_distribution = SparseCat(Observations, Observation_matrix[s,:])
            obs = rand(obs_distribution)
            return (sp=s,r=r,o=obs)
        else
            crd = Categorical(T[s,:,2])
            s = rand(crd)
            r = normal_operation
            obs_distribution = SparseCat(Observations, Observation_matrix[s,:])
            obs = rand(obs_distribution)
            return (sp=s,r=r,o=obs)
        end
    end

    if a==3 # Maintenance occurs in the system, no system failure, preventive maintenance for the unit
        if s==Number_level
            crd=Categorical(T[1,:,3])
            s=rand(crd)
            r=failure_penalty  + normal_operation
            obs_distribution = SparseCat(Observations, Observation_matrix[s,:])
            obs = rand(obs_distribution)
            return (sp=s,r=r,o=obs)
        else
            crd = Categorical(T[s,:,3])
            s = rand(crd)
            r = maintenance_penalty+normal_operation
            obs_distribution = SparseCat(Observations, Observation_matrix[s,:])
            obs = rand(obs_distribution)
            return (sp=s,r=r,o=obs)
        end
    end

    if a==4 # maintenance in system, system failure occurs, preventive replacement for the unit
        if s==Number_level
            crd = Categorical(T[1,:,3])
            s = rand(crd)
            r=failure_penalty+ normal_operation
            obs_distribution = SparseCat(Observations, Observation_matrix[s,:])
            obs = rand(obs_distribution)
            return (sp=s,r=r,o=obs)
        else
            r = maintenance_penalty+normal_operation
            crd = Categorical(T[1,:,3])
            s = rand(crd)
            obs_distribution = SparseCat(Observations, Observation_matrix[s,:])
            obs = rand(obs_distribution)
            return (sp=s,r=r,o=obs)
        end
    end
end

        
        
    

generative (generic function with 2 methods)

#### Belief updater

$$b'(s_j)=P(s_j\vert o,a,b)=\dfrac{P(o\vert s_j,a)\sum_{s_i\in \mathcal{S}}P(s_j\vert s_i,a)b(s_i)}{\sum_{s_j\in \mathcal{S}}P(o\vert s_j,a)\sum_{s_i\in \mathcal{S}}P(s_j\vert s_i,a)b(s_i)}$$

In [34]:
function update_belief(b,a,o,pomdp) 
    # NOTE: 
    # in input, b is belief from last epoch (before a)
    # a is the last action we took
    # o is the new observation after a
    updated_belief = zeros(length(b))
    if o==4
        updated_belief[length(states(pomdp))]=1.0
        return updated_belief
    end
    #if ((a==2)&&(o==4)) ||(a==3) ||(a==4)
    #if (a==2) ||(a==3) ||(a==4)
    if (a==3) ||(a==4)
    # We assume perfect maintenance and it is know to decision maker(belief updater)
        for sp in 1: length(states(pomdp))
            p_observation = POMDPs.observation(pomdp,a,sp) 
            p_obs = pdf(p_observation,o)
            # Transition probability distribution is T[1,:,1]
            p_trans = pdf(transition(pomdp,1,1),sp) 
            updated_belief[sp] = p_obs * p_trans
        end
        updated_belief ./= sum(updated_belief)
        return updated_belief
    end
    
    for sp in 1:length(states(pomdp)) 
        p_observation = POMDPs.observation(pomdp,a,sp) 
        p_obs = pdf(p_observation,o)
        p_trans = 0.0
        for s in 1:length(states(pomdp))
            #trans = transition(pomdp, s,a)
            #p_trans += pdf(transition(pomdp, s,a),sp) * pdf(b, s)
            # p_trans += T[s,sp,a] * pdf(b, s)
            p_trans += pdf(transition(pomdp,s,a),sp) * b[s]
        end
        updated_belief[sp] = p_obs * p_trans
    end
        # Normalize the updated belief state to ensure it sums up to 1
        updated_belief ./= sum(updated_belief)
    return updated_belief
end


update_belief (generic function with 1 method)

### Value Iteration

In [35]:
function ValueIteration(S, T, γ)
    # Here we always consider the state with the highest belief as the true state
    Q = zeros((Number_level, 4))
    V = zeros(Float64, (Number_level)).+1000
    i=0
    while i<=10000
        for s in 1:Number_level
            Threads.@threads for a in 1:4
                # Q[s,a] = (reward(s,a)) + γ*(sum(T[s,s1,a]*V[s1] for s1 in 1:Number_level))
                # reward(multiunit, 2,2)
                if a == 4
                    Q[s,a] = (reward(multiunit,s,a)) + γ*(sum(T[s,s1,3]*V[s1] for s1 in 1:Number_level))
                else
                Q[s,a] = (reward(multiunit,s,a)) + γ*(sum(T[s,s1,a]*V[s1] for s1 in 1:Number_level))
                end
            end
            V[s] = findmax(Q[s, 1:4])[1]
        end
        i+=1
    end
    return Q
end

ValueIteration (generic function with 1 method)

### System-level decision making

In [36]:
function decision(Q, o, b)
    a_final = repeat(1:1,NumberUnits)
    # Instead of using maximum likelihood state,
    # we use belief-weighted sum of Q function

    failed_components = 0
    for i in 1:NumberUnits # numerate over units
        if o[i] == Number_level # failure state is self-announcing
            failed_components+=1
        end
    end

    # calculate belief-based Q table for each unit
    Q_belief = zeros((NumberUnits, 4))
    
    for i in 1:NumberUnits
        belief = b[i]
        for a in 1:4
            Q_belief[i,a] = sum(belief[s] * Q[s,a] for s in 1:Number_level)
        end
    end

    # decision making: 3 scenarios
    # 1) System fails
    if failed_components >= limit
        for i in 1:NumberUnits
            a_final[i]=argmax(a->Q_belief[i,a],[2,3])
            if o[i] == Number_level
                a_final[i]=4
            end
        end
        return a_final
    end

    q0 = 0.0
    a0 = repeat(1:1, NumberUnits)
    for i in 1:NumberUnits
        q0+=Q_belief[i,1]
    end

    q1 = 0.0
    a1 = repeat(1:1, NumberUnits)
    for i in 1:NumberUnits
        a1[i] = argmax(a->Q_belief[i,a], [2,3])
        if o[i] == Number_level
            a1[i] = 3
        end
        q1 += Q_belief[i, a1[i]]
    end

    if !(3 in a1)
        a1 = a0
        q1 = q0
    end

    if q0 > q1
        return a0
    else
        return a1
    end
end
    



decision (generic function with 1 method)

In [37]:
# function decision(Q, o, b)
#     a_final = repeat(1:1,NumberUnits)
#     # obtain "guessed" state vector from belief vector
#     # and consider it as the true state vector
#     s_guess = argmax.(b)

#     failed_components = 0
#     for i in 1:NumberUnits # numerate over units
#         if o[i] == Number_level # failure state is self-announcing
#             failed_components+=1
#         end
#     end

#     if failed_components >= limit
#         for i in 1:NumberUnits
#             a_final[i]=argmax(a->Q[s_guess[i],a],[2,3])
#             if o[i] == Number_level
#                 a_final[i]=4
#             end
#         end
#         return a_final
#     end

#     q0 = 0.0
#     a0 = repeat(1:1, NumberUnits)
#     for i in 1:NumberUnits
#         q0+=Q[s_guess[i],1]
#     end

#     q1 = 0.0
#     a1 = repeat(1:1, NumberUnits)
#     for i in 1:NumberUnits
#         a1[i] = argmax(a->Q[s_guess[i],a], [2,3])
#         if o[i] == Number_level
#             a1[i] = 3
#         end
#         q1 += Q[s_guess[i], a1[i]]
#     end

#     if !(3 in a1)
#         a1 = a0
#         q1 = q0
#     end

#     if q0 > q1
#         return a0
#     else
#         return a1
#     end
# end
    


   
    
    

### Iteration

In [38]:
NumberUnits = 10
belief = [[1.0, 0.0, 0.0, 0.0] for j in 1:NumberUnits]

10-element Vector{Vector{Float64}}:
 [1.0, 0.0, 0.0, 0.0]
 [1.0, 0.0, 0.0, 0.0]
 [1.0, 0.0, 0.0, 0.0]
 [1.0, 0.0, 0.0, 0.0]
 [1.0, 0.0, 0.0, 0.0]
 [1.0, 0.0, 0.0, 0.0]
 [1.0, 0.0, 0.0, 0.0]
 [1.0, 0.0, 0.0, 0.0]
 [1.0, 0.0, 0.0, 0.0]
 [1.0, 0.0, 0.0, 0.0]

In [39]:
belief[1]=[0.7,0.3,0.0,0.0]

4-element Vector{Float64}:
 0.7
 0.3
 0.0
 0.0

In [40]:
belief

10-element Vector{Vector{Float64}}:
 [0.7, 0.3, 0.0, 0.0]
 [1.0, 0.0, 0.0, 0.0]
 [1.0, 0.0, 0.0, 0.0]
 [1.0, 0.0, 0.0, 0.0]
 [1.0, 0.0, 0.0, 0.0]
 [1.0, 0.0, 0.0, 0.0]
 [1.0, 0.0, 0.0, 0.0]
 [1.0, 0.0, 0.0, 0.0]
 [1.0, 0.0, 0.0, 0.0]
 [1.0, 0.0, 0.0, 0.0]

In [41]:
b = belief[1,:]

1-element Vector{Vector{Float64}}:
 [0.7, 0.3, 0.0, 0.0]

In [42]:
b[1]

4-element Vector{Float64}:
 0.7
 0.3
 0.0
 0.0

In [43]:
# Define the fixed K values for each unit count
function get_fixed_K(units)
    if units == 60
        return 18
    elseif units == 50
        return 18
    elseif units == 40
        return 18
    elseif units == 30
        return 14
    elseif units == 25
        return 10  # Assuming a reasonable value for 25
    elseif units == 20
        return 10   # Assuming a reasonable value for 20
    else
        return convert(Int64, floor((units+1)/4))  # Fallback for any other values
    end
end

get_fixed_K (generic function with 1 method)

In [44]:
using DataFrames
observation_matrices = [
    # Low Uncertainty
    [
        0.9  0.1  0.0  0.0;
        0.05 0.9  0.05 0.0;
        0.0  0.1  0.9  0.0;
        0.0  0.0  0.0  1.0
    ],
    # Original Uncertainty
    [
        0.8  0.2  0.0  0.0;
        0.1  0.8  0.1  0.0;
        0.0  0.2  0.8  0.0;
        0.0  0.0  0.0  1.0
    ],
    # High Uncertainty
    [
        0.7  0.3  0.0  0.0;
        0.15 0.7  0.15 0.0;
        0.0  0.3  0.7  0.0;
        0.0  0.0  0.0  1.0
    ],
    # Very High Uncertainty
    [
        0.6  0.4  0.0  0.0;
        0.2  0.6  0.2  0.0;
        0.0  0.4  0.6  0.0;
        0.0  0.0  0.0  1.0
    ],
    # Extreme Uncertainty
    [
        0.5  0.5  0.0  0.0;
        0.25 0.5  0.25 0.0;
        0.0  0.5  0.5  0.0;
        0.0  0.0  0.0  1.0
    ]
]

# Labels for the observation matrices (useful for output and plotting)
uncertainty_labels = ["Low", "Original", "High", "Very High", "Extreme"]

# Modified DataFrame with an additional column for uncertainty level
df = DataFrame(uncertainty=[], u=[], K=[], n=[], s=[], m=[], f=[], p=[], lamb=[], mean=[], std=[])

cost=[ 
    [0  -2400   -100    -1200 -2000]

] 


for units in [20, 25, 30, 40, 50, 60]

    global limit = get_fixed_K(units)
    global K = limit
        
    global NumberUnits=convert(Int64,units)
 
    #for lambda in [1,0.9,0.75,0.5,0.1,0.075,0.05,0.01,0.0075,0.005,0.0001]
    for lambda in 1:1:1
    global lamb=lambda
    for c in cost
        for obs_idx in 1:length(observation_matrices)
            global Observation_matrix = observation_matrices[obs_idx]
            global uncertainty_level = uncertainty_labels[obs_idx]
        tick()
        println(units)  
        print("K : ")
        println(K)
        global normal_operation,setup_cost,maintenance_penalty,failure_penalty,system_penalty=c
        println(c)  
        println("Uncertainty Level: ", uncertainty_level)

        global NumberUnits=units

# POMDP is set up
# CONTINUE to obtain TRIVIAL SOLUTION and compute expected total rewards

# Obtain Q function using Value Iteration, same as CW-MDP
Q = ValueIteration(1:Number_level, T, gamma)
println(isnan(Q[1,1]))
if sum(isnan(Q[1,1])) > 0.0
    continue
end

tick()

trials=10000;
simsteps = 100;
results=zeros(trials)*0.1
Threads.@threads  for h in 1:trials
# Suppose we start from brand new states and we know it
s=repeat(1:1,NumberUnits)
# o=s 
o = rand(Initialobs, NumberUnits)
# belief = [[1.0, 0.0, 0.0, 0.0] for j in 1:NumberUnits]
belief = [Observation_matrix[1,:] for j in 1:NumberUnits]
R_sequence=[] 
r=repeat(1:1,NumberUnits)*1.0
# a=repeat(1:1,NumberUnits)

# iteration
for k in 1:simsteps
    # function decision(Q, o, b)
 
    a=decision(Q,o,belief) # system-level decision making: action vector
    # SANITY CHECK
    # Update belief for each component individually
    #b=update_belief(b, a, o, multiunit) # what is the sequence of observing, updating belief, taking action?
    
    R_instant = 0.0
    global failed_components=0
    # When generating new state and new reward, we use true states
    for i in 1:NumberUnits # numerate over units
        if o[i]==Number_level # failure state is self-announcing
            failed_components+=1
        end
    s[i],r[i],o[i]=generative(s[i],a[i],234) #call generative function for underlying states
    # it might be better to use generative function to give o as well
    # o[i]=emission(s[i]) # observation generated by new underlying state
    belief[i]=update_belief(belief[i], a[i], o[i], multiunit)
    # sum up single-component cost
    R_instant += r[i]
    end
    # add setup cost
    if (2 in a) ||(3 in a) ||(4 in a)
        R_instant += setup_cost
    end
    # add system failure penalty
    if failed_components >= limit
        R_instant += system_penalty
    end
    # record instant reward of current time step
    append!(R_sequence, R_instant)

end
R_discounted=0
for k in 1:simsteps
    R_discounted=gamma * R_discounted+R_sequence[simsteps-k+1]
end
results[h]=R_discounted
end # end the 10000 trials

println(units)
            print("k : ")
            println(limit)
println(lambda)

println(mean(results))

println(std(results))
println(std(results))
    pushfirst!(df, [
        uncertainty_level,  # Add the uncertainty level as the first column
        units, 
        limit, 
        normal_operation, 
        setup_cost, 
        maintenance_penalty, 
        failure_penalty, 
        system_penalty, 
        lamb, 
        mean(results), 
        std(results)
    ])
        
tock()

                    
                    
    end
    end        
end
end


20
K : 10


[ Info:  started timer at: 2025-04-11T17:10:01.048


[0 -2400 -100 -1200 -2000]
Uncertainty Level: Low
false


[ Info:  started timer at: 2025-04-11T17:10:06.318


20
k : 10
1
-13533.477251042223
1802.7784007743253
1802.7784007743253
20
K : 10
[0 -2400 -100 -1200 -2000]
Uncertainty Level: Original


...

[ Info:           75.9263239s: 1 minute, 15 seconds, 926 milliseconds
[ Info:  started timer at: 2025-04-11T17:34:19.527


false


[ Info:  started timer at: 2025-04-11T17:34:22.795


60
k : 18
1
-31819.72503322261
2638.723190356039
2638.723190356039


[ Info:           75.5712643s: 1 minute, 15 seconds, 571 milliseconds


In [45]:
df

Row,uncertainty,u,K,n,s,m,f,p,lamb,mean,std
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,Extreme,60,18,0,-2400,-100,-1200,-2000,1,-31819.7,2638.72
2,Very High,60,18,0,-2400,-100,-1200,-2000,1,-31292.9,2647.44
3,High,60,18,0,-2400,-100,-1200,-2000,1,-31766.2,2601.37
4,Original,60,18,0,-2400,-100,-1200,-2000,1,-31539.4,2548.05
5,Low,60,18,0,-2400,-100,-1200,-2000,1,-30813.8,2491.99
6,Extreme,50,18,0,-2400,-100,-1200,-2000,1,-27031.2,2461.35
7,Very High,50,18,0,-2400,-100,-1200,-2000,1,-26960.6,2416.05
8,High,50,18,0,-2400,-100,-1200,-2000,1,-27472.1,2434.83
9,Original,50,18,0,-2400,-100,-1200,-2000,1,-27396.9,2340.31


In [46]:
using CSV
CSV.write("CW-MDP.csv", df)

"CW-MDP.csv"